## Training on MNIST Dataset with Contrastive NTXent Loss
-------------------------

In [1]:
import os
#import torchsummary
import matplotlib.pyplot as plt
import torch
import numpy as np
import cv2
from torch import nn
from torch.utils.data import Dataset, DataLoader, SubsetRandomSampler
from torchvision import transforms, models

os.chdir('..'); os.chdir('..')
print(os.getcwd()) # Should be .\ContrastiveLearning
from Code.trainers import Trainer
#from Code.models import SiameseNet
from Code.losses import form_triplets, ContrastiveLoss
from Code.dataloaders import LabeledContrastiveDataset



# Hyperparameters
N = 3000
EMB_SIZE = 2
DEVICE = 'cuda'
LR = 0.0005
EPOCHS = 10
MARGIN = 1.0
NAME = 'MNIST_NTXent_LOSS_' + '_'.join([str(N), str(EMB_SIZE), str(LR), str(EPOCHS), str(MARGIN)])

# Reproduciblity
SEED = 911
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(SEED)

D:\Research\ContrastiveRepresentationLearning


## Create Dataloader and Inspect Data
---------------------

In [2]:
root = r'D:\Data\Imagery\MNIST\MNIST'
mean, std = 0.1307, 0.3081

tfms  =    transforms.Compose([
                                 transforms.ToTensor(),
                                 transforms.Normalize((mean,), (std,))
                             ])


lcd = LabeledContrastiveDataset(root, transforms=tfms)


In [3]:
datadict = lcd.__getitem__(4)
print(datadict["x1"].shape); print(datadict["x2"].shape)

torch.Size([10, 1, 28, 28])
torch.Size([10, 1, 28, 28])


C:\Users\Shair\.conda\envs\pytorch\lib\site-packages\torchvision\transforms\functional.py:114: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:143.)
  img = torch.from_numpy(pic.transpose((2, 0, 1))).contiguous()


In [4]:
from torchvision import transforms


train_sampler = SubsetRandomSampler(range(int(N*0.9)))
test_sampler = SubsetRandomSampler(range(int(N*0.9), N))

siamese_train_loader = torch.utils.data.DataLoader(lcd, batch_size=None, sampler=train_sampler)
siamese_test_loader = torch.utils.data.DataLoader(lcd, batch_size=None, shuffle=test_sampler)


## Model
------------

In [5]:
embedding_net = models.resnet18()
embedding_net.conv1 = nn.Conv2d(1, 64, (7,7), (2,2), (3,3))
embedding_net.fc = nn.Linear(512, EMB_SIZE)
model = embedding_net
model.train(); print() ; #torchsummary.summary(model, input_size = [(1,28,28),(1, 28, 28)], device=DEVICE)

## Training
-------------------------

In [6]:

TL = ContrastiveLoss(margin=1.0, mode='ntxent', batch_size = 10)

t = Trainer(model = model, 
            dataloader = siamese_train_loader,
            lr=LR,
            loss_function= TL)

In [7]:
losses = t.train(EPOCHS, print_every=1)#, writer = writer)

  0%|                                                                                         | 0/2700 [00:00<?, ?it/s]

----- Epoch: 0 -----


  0%|                                                                                         | 0/2700 [00:01<?, ?it/s]


RuntimeError: Expected all tensors to be on the same device, but found at least two devices, cuda:0 and cpu!

In [ ]:
plt.plot(losses)
plt.title("Training Loss")
plt.ylabel("Train loss"); plt.xlabel("Epochs")


In [ ]:
losses

## Inspecting Embeddings
-------------------

In [ ]:
EMBS_TO_VISUALIZE = N - int(N*0.9)
mnist_classes = [0,1,2,3,4,5,6,7,8,9]
colors = ['#1f77b4', '#ff7f0e', '#2ca02c', '#d62728',
              '#9467bd', '#8c564b', '#e377c2', '#7f7f7f',
              '#bcbd22', '#17becf']

coloring = {mnist_classes[i]: colors[i] for i in range(len(mnist_classes))}

def extract_embeddings(dl, model, n, device='cuda'):
    
    import pandas as pd
    
    model.eval()
    m = model.to(device)
    
    embs = None
    labs = None
    tot_cnt = 0
    
    for idx, dat in enumerate(dl):
        
        # Extract relevent data points from batch
        x1 = dat['x1'] # (B,C,W,H)
        x2 = dat['x2'] # (B,C,W,H)
        labels = dat['labels'].type(torch.LongTensor) # (B,1)
        
        # Send inputs to correct device
        x1 = x1.to(device); x2 = x2.to(device)
        
        # Pass inputs through model
        emb1 = m(x1); emb2 = m(x2) #(B, EMB_SIZE)
        
        # Add to running lists
        if embs is None:
            embs = torch.cat([emb1, emb2])
        else:
            embs = torch.cat([embs, emb1, emb2])
        
        # Add coloring to list
        if labs is None:
            labs = torch.cat([labels, labels])
        else:
            labs = torch.cat([labs, labels, labels])

        # Check count of points so far
        curr_cnt = x1.shape[0] + x2.shape[0]
        if tot_cnt + curr_cnt >= n:
            break
        tot_cnt += curr_cnt
    
    # Convert to numpy array and seperate coords for plotting
    embs = [np.array(e.detach()) for e in embs]
    embs_x = [e[0] for e in embs]; embs_y = [e[1] for e in embs]
    labs = np.array([x.item() for x in labs])
    
    df = pd.DataFrame({"X": embs_x, "Y": embs_y, "Label": labs})
    
    return(df)
    
    return((embs_x, embs_y), labs)



In [ ]:
test_embs = extract_embeddings(siamese_test_loader, model, EMBS_TO_VISUALIZE, 'cpu')

In [ ]:
test_embs.head()

In [ ]:
import seaborn as sns
sns.set_style("darkgrid")
sns.relplot(x="X", y="Y", data=test_embs, hue="Label", palette="deep", alpha=0.7, s=75)
plt.title("Test Embeddings")

## Saving Model
-------------------------

In [ ]:
outpath = os.getcwd() + r'\Outputs\Weights\' + NAME 
torch.save(model.state_dict(), outpath); print(outpath)